In [1]:
import duckdb
import shutil
import os, sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import utils 
import warnings
warnings.filterwarnings('ignore')

## Join two sources into one table

In [2]:

nationalites = utils.DBtable_to_df('../integration.duckdb', 'nationalitiesClean_Madrid')
households = utils.DBtable_to_df('../integration.duckdb','householdClean_Madrid')

df = households.merge(nationalites, how='inner', on=["Madrid_section","Year"])

utils.df_to_DBtable('../integration.duckdb',df, 'integratedTable')

### Test

In [3]:
# check if all tables in integration.duckdb are created
utils.get_tables('../integration.duckdb')

0    nationalitiesClean_Madrid
1        householdClean_Madrid
2              integratedTable
Name: table_name, dtype: object

In [4]:
df

,Madrid_section,single_women_aged_16_to_64,single_men_aged_16_to_64,single_women_aged_65_or_over,single_men_aged_65_or_over,adult_women_with_one_or_more_minors,adult_men_with_one_or_more_minors,two_adults_from_16_to_64_and_without_minors,two_adults_one_at_least_65_and_without_minors,two_adults_and_one_minor,...,Ucrania,Uganda,Uruguay,Uzbekistán,Vanuatu,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
0,079601001,67.0,88.0,62.0,16.0,9.0,4.0,101.0,69.0,15.0,...,2.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
1,079601002,99.0,89.0,36.0,15.0,5.0,3.0,84.0,29.0,11.0,...,0.0,0.0,3.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0
2,079601003,147.0,151.0,71.0,42.0,7.0,1.0,144.0,75.0,23.0,...,7.0,0.0,4.0,0.0,0.0,22.0,1.0,0.0,0.0,0.0
3,079601004,99.0,95.0,55.0,20.0,9.0,1.0,88.0,71.0,23.0,...,1.0,1.0,2.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0
4,079601006,142.0,174.0,75.0,42.0,12.0,1.0,151.0,80.0,33.0,...,6.0,0.0,2.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7324,079621029,32.0,29.0,6.0,5.0,20.0,12.0,45.0,19.0,53.0,...,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
7325,079621030,37.0,46.0,29.0,11.0,19.0,6.0,68.0,52.0,44.0,...,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7326,079621031,88.0,91.0,15.0,6.0,39.0,9.0,156.0,31.0,153.0,...,7.0,0.0,0.0,0.0,0.0,26.0,0.0,0.0,0.0,0.0
7327,079621032,65.0,71.0,17.0,4.0,27.0,13.0,82.0,23.0,68.0,...,0.0,0.0,2.0,1.0,0.0,9.0,0.0,0.0,0.0,0.0


## Removing redundacies from integrated table

### Removing columns with zeros in all rows

In [5]:
df = utils.DBtable_to_df('../integration.duckdb', 'integratedTable')
for (columnName, columnData) in df.items():
    if(df[columnName] == 0).all():
        df.drop(columnName, axis=1, inplace=True)
        print(columnName)
display(df)


Brunei
Maldivas
Santa_Sede
Swazilandia
Vanuatu


,Madrid_section,single_women_aged_16_to_64,single_men_aged_16_to_64,single_women_aged_65_or_over,single_men_aged_65_or_over,adult_women_with_one_or_more_minors,adult_men_with_one_or_more_minors,two_adults_from_16_to_64_and_without_minors,two_adults_one_at_least_65_and_without_minors,two_adults_and_one_minor,...,Túnez,Ucrania,Uganda,Uruguay,Uzbekistán,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
0,079601001,67.0,88.0,62.0,16.0,9.0,4.0,101.0,69.0,15.0,...,0.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
1,079601002,99.0,89.0,36.0,15.0,5.0,3.0,84.0,29.0,11.0,...,0.0,0.0,0.0,3.0,0.0,13.0,0.0,0.0,0.0,0.0
2,079601003,147.0,151.0,71.0,42.0,7.0,1.0,144.0,75.0,23.0,...,0.0,7.0,0.0,4.0,0.0,22.0,1.0,0.0,0.0,0.0
3,079601004,99.0,95.0,55.0,20.0,9.0,1.0,88.0,71.0,23.0,...,0.0,1.0,1.0,2.0,0.0,11.0,0.0,0.0,0.0,0.0
4,079601006,142.0,174.0,75.0,42.0,12.0,1.0,151.0,80.0,33.0,...,0.0,6.0,0.0,2.0,0.0,13.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7324,079621029,32.0,29.0,6.0,5.0,20.0,12.0,45.0,19.0,53.0,...,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
7325,079621030,37.0,46.0,29.0,11.0,19.0,6.0,68.0,52.0,44.0,...,0.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7326,079621031,88.0,91.0,15.0,6.0,39.0,9.0,156.0,31.0,153.0,...,0.0,7.0,0.0,0.0,0.0,26.0,0.0,0.0,0.0,0.0
7327,079621032,65.0,71.0,17.0,4.0,27.0,13.0,82.0,23.0,68.0,...,0.0,0.0,0.0,2.0,1.0,9.0,0.0,0.0,0.0,0.0


### Checking if there is section code with all zeros in remaining columns

In [6]:

display(df[df.columns[1:]].loc[(df[df.columns[1:]]== 0).all(axis=1)])
# none are found

,single_women_aged_16_to_64,single_men_aged_16_to_64,single_women_aged_65_or_over,single_men_aged_65_or_over,adult_women_with_one_or_more_minors,adult_men_with_one_or_more_minors,two_adults_from_16_to_64_and_without_minors,two_adults_one_at_least_65_and_without_minors,two_adults_and_one_minor,two_adults_and_two_minors,...,Túnez,Ucrania,Uganda,Uruguay,Uzbekistán,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe


## Saving copy of integrated table to analysis folder

In [7]:
utils.df_to_DBtable('../integration.duckdb',df, 'integratedTable')
shutil.copy("../integration.duckdb","../../2.Data Analysis Backbone/")
print('Copied')

Copied
